### Comparer deux modèles (LogisticRegression et SVC)
1. corriger les warnings
2. Logger la matrice de confusion comme artefact

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from mlflow.models.signature import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Chargement des données
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Crée un dossier pour stocker les images si besoin
os.makedirs("plots", exist_ok=True)

# Liste des modèles à comparer
models = {
    "logreg_2": LogisticRegression(max_iter=200, random_state=42),
    "svc_2": SVC(kernel="linear", probability=True, random_state=42)
}

# Si un run est encore actif (en Notebook), le fermer
if mlflow.active_run():
    mlflow.end_run()

# Comparaison
for name, clf in models.items():
    with mlflow.start_run(run_name=name):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Log des paramètres et métriques
        mlflow.log_param("model", name)
        mlflow.log_metric("accuracy", acc)

        # Signature et exemple pour éviter les warnings
        input_example = X_test[:1]
        signature = infer_signature(X_test, y_pred)

        # Log du modèle avec signature
        mlflow.sklearn.log_model(clf, "model", signature=signature, input_example=input_example)

        # Création + log de la matrice de confusion
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
        plt.title(f"Confusion Matrix - {name}")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        fig_path = f"plots/cm_{name}.png"
        plt.tight_layout()
        plt.savefig(fig_path)
        mlflow.log_artifact(fig_path)
        plt.close()

print("✅ Comparaison terminée ! Lance `mlflow ui` et visite http://localhost:5000 pour explorer tes modèles.")


✅ Comparaison terminée ! Lance `mlflow ui` et visite http://localhost:5000 pour explorer tes modèles.
